In [1]:
## OK Arreglar metricas (f1, precision, etc), en acurracy quedarte con la mejor de las epocas
## OK Implementar la clase weight
## OK Implementar Cobyla
## OK "rep_fm": [1, 2],  # Explorar "rep_ans": [2, 3],  # Explorar
## OK Aumentar iteraciones
## OK ejecutar sin smote
## ejecutar con smote
## hacer lo que pidió la profe
## Implementar NLocal

In [2]:
!pip -q install scikit-learn pandas matplotlib seaborn
!pip install "qiskit==1.4.2" "qiskit-machine-learning==0.8.2"

In [3]:
!pip -q install "qiskit-algorithms==0.3.*"   # versión compatible con Qiskit 1.4
!pip install -q qiskit-aer imblearn

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

# ─────────────────────────────── 1) Carga del CSV ───────────────────────────────
FILE_ID = "1a7DndWDsvbyKs1USmWtOp2Q0dbvCVb4n"
url     = f"https://drive.google.com/uc?export=download&id={FILE_ID}"

# sin cabecera; seleccionamos columnas 3..20  (18 parámetros + etiqueta)
df = pd.read_csv(url).iloc[:, 3:]

# ─────────────────────────────── 2) Separación X / y ────────────────────────────
X = df.iloc[:, :-1].values
y = df.iloc[:,  -1].astype(int).values        # asegúrate de int (0/1)

# Escalado (las features ya están en [0,1], pero se deja por buenas prácticas)
X = MinMaxScaler().fit_transform(X)

# ─────────────────────────────── 3) Oversampling SMOTE ──────────────────────────
#smote = SMOTE(random_state=42, k_neighbors=5)
#X_bal, y_bal = smote.fit_resample(X, y)
#print("Distribución tras SMOTE:", dict(pd.Series(y_bal).value_counts()))

# ─────────────────────────────── 4) Train/Test split ────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
# Para implementar la cross-entropy ponderada,
# lo mejor es calcular los pesos a partir de la distribución en el conjunto de entrenamiento
#(no de todo el dataset)
# clases únicas
classes = np.unique(y_train)            # [0, 1]
# calcula peso equilibrado: total/(n_classes * count_i)
cw = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
cw = np.array(cw, dtype=float)
# -- --- normalización dividiendo por la media -- --
# Con esto cw_norm.mean() == 1 y se seguirá penalizando la clase minoritaria
# ~10.6× más que la mayoritaria pero los gradientes tendrán
#una magnitud “familiar” y será necesario tocar la tasa de aprendizaje
cw_norm = cw

# devuelve algo como array([0.543, 6.279]) si la cl0=92% y cl1=8%
#class_weight_dict = {cls: weight for cls, weight in zip(classes, cw_norm)}
class_weight_list = [cw_norm[np.where(classes == cls)[0][0]] for cls in classes]

print("Pesos originales: ", dict(zip(classes, cw)))
print("Pesos normalizados: ", class_weight_list)
print(pd.Series(y_train).value_counts(normalize=True))

print("Shapes  ─ X_train:", X_train.shape,
      "| y_train:", y_train.shape,
      "| X_test:", X_test.shape,
      "| y_test:", y_test.shape)

# ─────────────────────────────── 5) PCA → N_QUBITS ──────────────────────────────
#N_QUBITS = 4
#pca = PCA(n_components=N_QUBITS, random_state=42)
#X_train_q = pca.fit_transform(X_train)
#X_test_q  = pca.transform(X_test)
#print("PCA shapes ─ X_train_q:", X_train_q.shape, "| X_test_q:", X_test_q.shape)

# ─────────────────────────────── 5) TOP3  ──────────────────────────────
# top-3
cols3 = ["vconst_corr", "vconst_2", "bckgrnd_vdc1"]
# Supongamos que 'df_full' es tu DataFrame con todas las features (antes de .values)
feature_cols = df.columns[:-1]   # todas menos 'outcome'
print(feature_cols.tolist())

# 2) Averigua los índices dentro de df.columns (excluyendo la etiqueta)
feature_cols = df.columns[:-1]  # todas las columnas excepto 'outcome'
col_idxs     = [list(feature_cols).index(c) for c in cols3]
print("Mapeo columnas → posiciones:", dict(zip(cols3, col_idxs)))

# 3) Tras tu train_test_split sobre arrays, filtra así:
N_QUBITS  = len(col_idxs)  # = 3
X_train_q = X_train[:, col_idxs]
X_test_q  = X_test[:, col_idxs]


Pesos originales:  {0: 5.837837837837838, 1: 0.5468354430379747}
Pesos normalizados:  [5.837837837837838, 0.5468354430379747]
1    0.914352
0    0.085648
Name: proportion, dtype: float64
Shapes  ─ X_train: (432, 18) | y_train: (432,) | X_test: (108, 18) | y_test: (108,)
['vconst_corr', 'vconst_2', 'vconst_3', 'vconst_4', 'vconst_5', 'vconst_7', 'ah_corr', 'ah_bolus', 'slm_corr', 'efficiency_factor', 'tidal_mix_max', 'vertical_decay_scale', 'convect_corr', 'bckgrnd_vdc1', 'bckgrnd_vdc_ban', 'bckgrnd_vdc_eq', 'bckgrnd_vdc_psim', 'Prandtl']
Mapeo columnas → posiciones: {'vconst_corr': 0, 'vconst_2': 1, 'bckgrnd_vdc1': 13}


In [5]:
from qiskit_machine_learning.utils.loss_functions.loss_functions import Loss
class WeightedCrossEntropyLoss(Loss):
    r"""
    Cross-entropy loss with class weights:

    .. math::
        L = -\sum_{i=0}^{C-1} w_i \, t_i \,\log(p_i),
    where w_i is the weight for class i.
    """

    def __init__(self, class_weight: np.ndarray):
        """
        :param class_weight: array of shape (C,), weight per class.
        """
        self.class_weight = np.array(class_weight, dtype=float)
        assert self.class_weight.ndim == 1, "class_weight debe ser un vector 1D"

    def _validate_shapes(self, predict, target):
        super()._validate_shapes(predict, target)
        # además comprobamos que el número de clases coincida:
        C = predict.shape[-1] if predict.ndim > 1 else 1
        assert self.class_weight.shape[0] == C, (
            f"class_weight debe tener longitud {C}, "
            f"pero tiene {self.class_weight.shape[0]}"
        )

    def evaluate(self, predict: np.ndarray, target: np.ndarray) -> np.ndarray:
        self._validate_shapes(predict, target)
        # aseguramos batch dimension:
        if predict.ndim == 1:
            predict = predict.reshape(1, -1)
            target  = target.reshape(1, -1)

        # clip y log
        log_p = np.log2(np.clip(predict, a_min=1e-10, a_max=None))

        # aplicamos el peso por clase al target
        # shape (batch, C) = (batch, C) * (C,)
        weighted_target = target * self.class_weight[np.newaxis, :]

        # -sum_i w_i * t_i * log p_i   para cada muestra
        losses = -np.einsum("ij,ij->i", weighted_target, log_p).reshape(-1, 1)
        return losses

    def gradient(self, predict: np.ndarray, target: np.ndarray) -> np.ndarray:
        self._validate_shapes(predict, target)
        if predict.ndim == 1:
            predict = predict.reshape(1, -1)
            target  = target.reshape(1, -1)

        # target ponderado
        weighted_target = target * self.class_weight[np.newaxis, :]

        # suma de w_j * t_j por fila: shape (batch,)
        sum_wt = np.sum(weighted_target, axis=1)

        # grad = p * (sum_j w_j t_j) - w * t
        grad = np.einsum("ij,i->ij", predict, sum_wt) - weighted_target
        return grad

In [6]:
# ──────────────────────────────────────────────────────────
# 0) Instalación e imports
# ──────────────────────────────────────────────────────────

import time, warnings, sys, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, EfficientSU2, RealAmplitudes, TwoLocal
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import SPSA
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    confusion_matrix,
    f1_score,
)
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler as AerSampler


weighted_loss_fn = WeightedCrossEntropyLoss(class_weight=class_weight_list)
# ──────────────────────────────────────────────────────────
# Parámetros de grid (solo SPSA por ahora)
# ──────────────────────────────────────────────────────────
params = {
    "loss_func_list"   : ["cross_entropy", "weightedcross_entropy", "absolute_error", "squared_error"],
    "feature_map_list" : ["ZFeatureMap", "ZZFeatureMap", "PauliFeatureMap"],
    "ansatz_list"      : ["EfficientSU2", "RealAmplitudes"],
    # repeticiones fijas
    "rep_fm_list"       : [1, 2],
    "rep_ans_list"      : [2, 3],
    # obligatorios para controlar iteraciones SPSA
    "spsa_maxiter"     : 30,
    "learning_rate_list": [0.05, 0.10],
}

# mapeos de nombre a clase
feature_map_classes = {
    "ZFeatureMap"     : ZFeatureMap,
    "ZZFeatureMap"    : ZZFeatureMap,
    "PauliFeatureMap" : PauliFeatureMap,
}
ansatz_classes = {
    "EfficientSU2"   : EfficientSU2,
    "RealAmplitudes" : RealAmplitudes,
    "TwoLocal" : TwoLocal,
}

loss_fn_map = {
    "weightedcross_entropy": weighted_loss_fn,
    "cross_entropy":         "cross_entropy",
    "absolute_error":        "absolute_error",
    "squared_error":         "squared_error",
}

grid_results = {
    "feature_map": [],
    "ansatz": [],
    "rep_fm": [],
    "rep_var": [],
    "learning_rate": [],
    "loss_func": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": [],
    "duration": [],
}

n_runs = len(params["loss_func_list"]) * len(params["feature_map_list"]) * \
         len(params["ansatz_list"]) * len(params["learning_rate_list"]   * \
         len(params["rep_fm_list"])  * len(params["rep_ans_list"]))

actual_run = 0
# ──────────────────────────────────────────────────────────
# 2) Grid search loop
# ──────────────────────────────────────────────────────────
for loss_name in params["loss_func_list"]:
    for fm_name in params["feature_map_list"]:
        for ans_name in params["ansatz_list"]:
          for rep_fm in params["rep_fm_list"]:       # <— nuevo bucle
            for rep_ans in params["rep_ans_list"]: # <— nuevo bucle
              for LR in params["learning_rate_list"]:

                actual_run += 1

                N_QUBITS  = 3
                REPS_FM   = rep_fm
                REPS_ANS  = rep_ans
                EPOCHS    = params["spsa_maxiter"]

                print("═"*90)
                print(f"State: {actual_run}/{n_runs}:")
                print("═"*90)
                print(f"\t\t Feature map: {fm_name}")
                print(f"\t\t Ansatz: {ans_name}")
                print(f"\t\t Reps FM: {REPS_FM}")
                print(f"\t\t Reps var layer: {REPS_ANS}")
                print(f"\t\t Learning rate: {LR}")
                print(f"\t\t Loss function: {loss_name}")
                print("═"*90, end="\n\n")
                sampler_inst = AerSampler(run_options={"shots": None})
                loss_arg = loss_fn_map.get(loss_name)

                # ──────────────────────────────────────────────────────────
                # 2.2) Instanciar circuitos y optimizador SPSA
                # ──────────────────────────────────────────────────────────
                feature_map = feature_map_classes[fm_name](feature_dimension=N_QUBITS, reps=REPS_FM)
                ansatz      = ansatz_classes[ans_name](num_qubits=N_QUBITS, reps=REPS_ANS)

                training_loss     = []
                training_accuracy = []
                epoch_log         = []

                def logger(it, weights, loss, _step, accepted):
                    if not accepted:
                        return
                    epoch = len(training_loss) + 1
                    preds = np.argmax(vqc._neural_network.forward(X_train_q, weights), axis=1)
                    acc   = accuracy_score(y_train, preds)
                    training_loss.append(loss)
                    training_accuracy.append(acc)
                    epoch_log.append((epoch, loss, acc))
                    print(f"{epoch:>3} | loss={loss:.4f} | acc={acc:.3f}")

                optimizer = SPSA(
                    maxiter       = EPOCHS,
                    learning_rate = LR,
                    perturbation  = 0.1,
                    callback      = logger,
                    blocking      = True,
                )

                vqc = VQC(
                    sampler      = sampler_inst,
                    feature_map  = feature_map,
                    ansatz       = ansatz,
                    optimizer    = optimizer,
                    loss         = loss_arg,
                )

                start = time.time()
                vqc.fit(X_train_q, y_train)
                duration = time.time() - start
                print("\n\n---------> Training duration: {}".format(
                      time.strftime("%H:%M:%S", time.gmtime(duration))
                ))
                # ──────────────────────────────────────────────────────────
                # 2.4) Estadística final
                # ──────────────────────────────────────────────────────────
                mean_acc = np.mean(training_accuracy)
                print(f"\n\t\tMean training accuracy in the {EPOCHS} epochs: {mean_acc:.4f}\n")

                # ──────────────────────────────────────────────────────────
                # 2.5) Plots de loss y accuracy
                # ──────────────────────────────────────────────────────────
                """plt.figure(figsize=(6,4))
                plt.title("Loss function during training")
                sns.lineplot(x=range(1, len(training_loss) + 1), y=training_loss, color="blue")
                plt.xlabel("Epoch #")
                plt.ylabel("Loss")
                plt.tight_layout()
                plt.show()

                plt.figure(figsize=(6,4))
                plt.title("Accuracy during training")
                sns.lineplot(x=range(1, len(training_accuracy) + 1), y=training_accuracy, color="orange")
                plt.xlabel("Epoch #")
                plt.ylabel("Accuracy")
                plt.tight_layout()
                plt.show()"""

                # ──────────────────────────────────────────────────────────
                # 2.6) Evaluación en test y métricas finales
                # ──────────────────────────────────────────────────────────
                y_test_np = np.asarray(y_test)
                y_pred    = vqc.predict(X_test_q)

                print(classification_report(
                    y_test_np,
                    y_pred,
                    target_names=["fail (0)", "success (1)"],
                    digits=4
                ))

                accuracy_test  = accuracy_score(y_test_np,   y_pred)
                # 3) Métricas promedio ponderado 
                precision_w = precision_score(y_test_np, y_pred, average='macro')
                recall_w    = recall_score(   y_test_np, y_pred, average='macro')
                f1_w        = f1_score(       y_test_np, y_pred, average='macro')

                cm = confusion_matrix(y_test_np, y_pred)
                print("\nConfusion matrix [[TN, FP], [FN, TP]]:\n", cm)
                print("\n")

                grid_results["feature_map"].append(fm_name)
                grid_results["ansatz"].append(ans_name)
                grid_results["rep_fm"].append(REPS_FM)
                grid_results["rep_var"].append(REPS_ANS)
                grid_results["learning_rate"].append(LR)
                grid_results["loss_func"].append(loss_name)
                grid_results["accuracy"].append(accuracy_test)
                grid_results["precision"].append(precision_w)
                grid_results["recall"].append(recall_w)
                grid_results["f1"].append(f1_w)
                grid_results["duration"].append(duration)

print("\nGrid results:\n")
evaluation = pd.DataFrame(grid_results)

evaluation = evaluation.sort_values(
    ["accuracy", "precision", "recall"],
    ascending=False
)

display(evaluation)



══════════════════════════════════════════════════════════════════════════════════════════
State: 1/192:
══════════════════════════════════════════════════════════════════════════════════════════
		 Feature map: ZFeatureMap
		 Ansatz: EfficientSU2
		 Reps FM: 1
		 Reps var layer: 2
		 Learning rate: 0.05
		 Loss function: cross_entropy
══════════════════════════════════════════════════════════════════════════════════════════

  1 | loss=0.8057 | acc=0.722
  2 | loss=0.8004 | acc=0.727
  3 | loss=0.7703 | acc=0.738
  4 | loss=0.7575 | acc=0.745
  5 | loss=0.7522 | acc=0.748
  6 | loss=0.7495 | acc=0.748
  7 | loss=0.7480 | acc=0.748
  8 | loss=0.7370 | acc=0.759
  9 | loss=0.7335 | acc=0.762
 10 | loss=0.7174 | acc=0.764
 11 | loss=0.7157 | acc=0.769
 12 | loss=0.7140 | acc=0.769
 13 | loss=0.6856 | acc=0.775
 14 | loss=0.6700 | acc=0.780
 15 | loss=0.6695 | acc=0.780
 16 | loss=0.6548 | acc=0.782
 17 | loss=0.6400 | acc=0.787
 18 | loss=0.6350 | acc=0.787
 19 | loss=0.6337 | acc=0.787


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

  1 | loss=1.1189 | acc=0.250
  2 | loss=1.0202 | acc=0.509
  3 | loss=0.9821 | acc=0.595
  4 | loss=0.9799 | acc=0.611
  5 | loss=0.9311 | acc=0.692
  6 | loss=0.9291 | acc=0.699
  7 | loss=0.9041 | acc=0.729
  8 | loss=0.8712 | acc=0.755
  9 | loss=0.8695 | acc=0.764
 10 | loss=0.8484 | acc=0.762
 11 | loss=0.8481 | acc=0.766
 12 | loss=0.8271 | acc=0.773
 13 | loss=0.8252 | acc=0.775
 14 | loss=0.8180 | acc=0.769
 15 | loss=0.8175 | acc=0.773
 16 | loss=0.7783 | acc=0.782
 17 | loss=0.7573 | acc=0.785
 18 | loss=0.7391 | acc=0.799
 19 | loss=0.7389 | acc=0.803
 20 | loss=0.7277 | acc=0.801
 21 | loss=0.7086 | acc=0.808
 22 | loss=0.6971 | acc=0.812
 23 | loss=0.6970 | acc=0.812
 24 | loss=0.6942 | acc=0.812
 25 | loss=0.6941 | acc=0.812
 26 | loss=0.6897 | acc=0.810
 27 | loss=0.6893 | acc=0.810
 28 | loss=0.6826 | acc=0.812
 29 | loss=0.6775 | acc=0.822
 30 | loss=0.6699 | acc=0.831


---------> Training duration: 00:00:13

		Mean training accuracy in the 30 epochs: 0.7445

       

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

  1 | loss=0.8755 | acc=0.637
  2 | loss=0.8356 | acc=0.671
  3 | loss=0.8356 | acc=0.671
  4 | loss=0.8279 | acc=0.681
  5 | loss=0.7749 | acc=0.725
  6 | loss=0.7173 | acc=0.782
  7 | loss=0.7130 | acc=0.782
  8 | loss=0.6960 | acc=0.799
  9 | loss=0.6958 | acc=0.799
 10 | loss=0.6945 | acc=0.799
 11 | loss=0.6943 | acc=0.799
 12 | loss=0.6642 | acc=0.843
 13 | loss=0.6583 | acc=0.840
 14 | loss=0.6536 | acc=0.847
 15 | loss=0.6517 | acc=0.854
 16 | loss=0.6517 | acc=0.854
 17 | loss=0.6502 | acc=0.854
 18 | loss=0.6494 | acc=0.854
 19 | loss=0.6422 | acc=0.859
 20 | loss=0.6338 | acc=0.861
 21 | loss=0.6149 | acc=0.868
 22 | loss=0.6149 | acc=0.868
 23 | loss=0.6061 | acc=0.880
 24 | loss=0.5908 | acc=0.887
 25 | loss=0.5890 | acc=0.889
 26 | loss=0.5859 | acc=0.891
 27 | loss=0.5807 | acc=0.891
 28 | loss=0.5764 | acc=0.894
 29 | loss=0.5737 | acc=0.894


---------> Training duration: 00:00:14

		Mean training accuracy in the 30 epochs: 0.8197

              precision    recall  f1

...

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

  1 | loss=0.4905 | acc=0.542
  2 | loss=0.4620 | acc=0.572
  3 | loss=0.4354 | acc=0.609
  4 | loss=0.3424 | acc=0.787
  5 | loss=0.3111 | acc=0.859
  6 | loss=0.3064 | acc=0.863
  7 | loss=0.2975 | acc=0.873
  8 | loss=0.2747 | acc=0.894
  9 | loss=0.2698 | acc=0.896
 10 | loss=0.2681 | acc=0.896
 11 | loss=0.2665 | acc=0.898
 12 | loss=0.2596 | acc=0.898
 13 | loss=0.2559 | acc=0.900
 14 | loss=0.2550 | acc=0.903
 15 | loss=0.2549 | acc=0.903
 16 | loss=0.2454 | acc=0.907
 17 | loss=0.2426 | acc=0.912
 18 | loss=0.2411 | acc=0.910
 19 | loss=0.2407 | acc=0.910
 20 | loss=0.2404 | acc=0.910
 21 | loss=0.2403 | acc=0.910
 22 | loss=0.2403 | acc=0.910
 23 | loss=0.2396 | acc=0.912
 24 | loss=0.2395 | acc=0.912
 25 | loss=0.2389 | acc=0.914
 26 | loss=0.2385 | acc=0.914
 27 | loss=0.2375 | acc=0.914
 28 | loss=0.2366 | acc=0.914
 29 | loss=0.2358 | acc=0.914
 30 | loss=0.2358 | acc=0.914


---------> Training duration: 00:00:13

		Mean training accuracy in the 30 epochs: 0.8657

       

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

  1 | loss=0.4898 | acc=0.581
  2 | loss=0.4897 | acc=0.581
  3 | loss=0.4884 | acc=0.593
  4 | loss=0.4880 | acc=0.600
  5 | loss=0.4867 | acc=0.593
  6 | loss=0.4864 | acc=0.593
  7 | loss=0.4860 | acc=0.590
  8 | loss=0.4845 | acc=0.604
  9 | loss=0.4826 | acc=0.606
 10 | loss=0.4825 | acc=0.609
 11 | loss=0.4812 | acc=0.611
 12 | loss=0.4805 | acc=0.611
 13 | loss=0.4797 | acc=0.613
 14 | loss=0.4795 | acc=0.611
 15 | loss=0.4794 | acc=0.611
 16 | loss=0.4788 | acc=0.613
 17 | loss=0.4786 | acc=0.616
 18 | loss=0.4784 | acc=0.616
 19 | loss=0.4778 | acc=0.613
 20 | loss=0.4771 | acc=0.616
 21 | loss=0.4771 | acc=0.618
 22 | loss=0.4767 | acc=0.620
 23 | loss=0.4764 | acc=0.620
 24 | loss=0.4760 | acc=0.623
 25 | loss=0.4748 | acc=0.620
 26 | loss=0.4738 | acc=0.632
 27 | loss=0.4721 | acc=0.639
 28 | loss=0.4720 | acc=0.639
 29 | loss=0.4718 | acc=0.639
 30 | loss=0.4718 | acc=0.639


---------> Training duration: 00:00:14

		Mean training accuracy in the 30 epochs: 0.6123

       

,feature_map,ansatz,rep_fm,rep_var,learning_rate,loss_func,accuracy,precision,recall,f1,duration
146,ZFeatureMap,EfficientSU2,1,3,0.05,squared_error,0.944444,0.875728,0.717172,0.770863,13.647756
50,ZFeatureMap,EfficientSU2,1,3,0.05,weightedcross_entropy,0.925926,0.758704,0.858586,0.797563,13.833588
51,ZFeatureMap,EfficientSU2,1,3,0.10,weightedcross_entropy,0.925926,0.758704,0.858586,0.797563,13.960713
10,ZFeatureMap,RealAmplitudes,1,3,0.05,cross_entropy,0.916667,0.712264,0.550505,0.568958,12.317431
1,ZFeatureMap,EfficientSU2,1,2,0.10,cross_entropy,0.916667,0.458333,0.500000,0.478261,12.891154
...,...,...,...,...,...,...,...,...,...,...,...
88,PauliFeatureMap,RealAmplitudes,1,2,0.05,weightedcross_entropy,0.509259,0.478276,0.429293,0.382058,13.556021
90,PauliFeatureMap,RealAmplitudes,1,3,0.05,weightedcross_entropy,0.472222,0.518750,0.560606,0.393079,13.897192
138,PauliFeatureMap,RealAmplitudes,1,3,0.05,absolute_error,0.472222,0.472222,0.409091,0.361345,14.047859
171,ZZFeatureMap,RealAmplitudes,1,3,0.10,squared_error,0.472222,0.472222,0.409091,0.361345,14.007853


In [7]:
# ──────────────────────────────────────────────────────────
# COBYLA
# ──────────────────────────────────────────────────────────
from qiskit_machine_learning.optimizers import COBYLA

weighted_loss_fn = WeightedCrossEntropyLoss(class_weight=class_weight_list)
# ──────────────────────────────────────────────────────────
# Parámetros de grid COBYLA
# ──────────────────────────────────────────────────────────
params = {
    "loss_func_list"   : ["weightedcross_entropy", "cross_entropy", "absolute_error", "squared_error"],
    "feature_map_list" : ["ZFeatureMap", "ZZFeatureMap", "PauliFeatureMap"],
    "ansatz_list"      : ["EfficientSU2", "RealAmplitudes"],
    "rep_fm_list"       : [1, 2],
    "rep_ans_list"      : [2, 3],
    # obligatorios para controlar iteraciones SPSA
    "spsa_maxiter"     : 40,
    "learning_rate_list": [1e-3, 1e-4],
}

# mapeos de nombre a clase
feature_map_classes = {
    "ZFeatureMap"     : ZFeatureMap,
    "ZZFeatureMap"    : ZZFeatureMap,
    "PauliFeatureMap" : PauliFeatureMap,
}
ansatz_classes = {
    "EfficientSU2"   : EfficientSU2,
    "RealAmplitudes" : RealAmplitudes,
    "TwoLocal" : TwoLocal,
}

loss_fn_map = {
    "weightedcross_entropy": weighted_loss_fn,
    "cross_entropy":         "cross_entropy",
    "absolute_error":        "absolute_error",
    "squared_error":         "squared_error",
}

grid_results3 = {
    "feature_map": [],
    "ansatz": [],
    "rep_fm": [],
    "rep_var": [],
    "learning_rate": [],
    "loss_func": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": [],
    "duration": [],
}

n_runs = len(params["loss_func_list"]) * len(params["feature_map_list"]) * \
         len(params["ansatz_list"]) * len(params["learning_rate_list"]   * \
         len(params["rep_fm_list"])  * len(params["rep_ans_list"]))

actual_run = 0
# ──────────────────────────────────────────────────────────
# 2) Grid search loop
# ──────────────────────────────────────────────────────────
for loss_name in params["loss_func_list"]:
    for fm_name in params["feature_map_list"]:
        for ans_name in params["ansatz_list"]:
          for rep_fm in params["rep_fm_list"]:
            for rep_ans in params["rep_ans_list"]:
              for LR in params["learning_rate_list"]:

                actual_run += 1

                N_QUBITS  = 3
                REPS_FM   = rep_fm
                REPS_ANS  = rep_ans
                EPOCHS    = params["spsa_maxiter"]

                print("═"*90)
                print(f"State: {actual_run}/{n_runs}:")
                print("═"*90)
                print(f"\t\t Feature map: {fm_name}")
                print(f"\t\t Ansatz: {ans_name}")
                print(f"\t\t Reps FM: {REPS_FM}")
                print(f"\t\t Reps var layer: {REPS_ANS}")
                print(f"\t\t Learning rate: {LR}")
                print(f"\t\t Loss function: {loss_name}")
                print("═"*90, end="\n\n")
                
                sampler_inst = AerSampler(run_options={"shots": None})
                loss_arg = loss_fn_map.get(loss_name)

                # ──────────────────────────────────────────────────────────
                # 2.2) Instanciar circuitos y optimizador SPSA
                # ──────────────────────────────────────────────────────────
                feature_map = feature_map_classes[fm_name](feature_dimension=N_QUBITS, reps=REPS_FM)
                ansatz      = ansatz_classes[ans_name](num_qubits=N_QUBITS, reps=REPS_ANS)

                training_loss     = []
                training_accuracy = []
                epoch_log         = []

                def logger(weights, loss_val):
                  """Se ejecuta en cada paso del optimizador."""
                  epoch  = len(epoch_log) + 1
                  # forward() → probas, tomamos clase con mayor probabilidad
                  logits = vqc._neural_network.forward(X_train_q, weights)
                  preds  = np.argmax(logits, axis=1)
                  acc    = accuracy_score(y_train, preds)
                  training_loss.append(loss_val)
                  training_accuracy.append(acc)
                  epoch_log.append((epoch, loss_val, acc))
                  print(f"{epoch:<3} | loss {loss_val:.5f} | train acc {acc:.4f}")

                optimizer = COBYLA(
                    maxiter = EPOCHS,
                    tol     = LR,
                )

                vqc = VQC(
                    sampler      = sampler_inst,
                    feature_map  = feature_map,
                    ansatz       = ansatz,
                    optimizer    = optimizer,
                    loss         = loss_arg,
                )

                vqc.callback = logger                # enlazamos el callback


                start = time.time()
                vqc.fit(X_train_q, y_train)
                duration = time.time() - start
                print("\n\n---------> Training duration: {}".format(
                      time.strftime("%H:%M:%S", time.gmtime(duration))
                ))
                # ──────────────────────────────────────────────────────────
                # 2.4) Estadística final
                # ──────────────────────────────────────────────────────────
                mean_acc = np.mean(training_accuracy)
                print(f"\n\t\tMean training accuracy in the {EPOCHS} epochs: {mean_acc:.4f}\n")

                # ──────────────────────────────────────────────────────────
                # 2.5) Plots de loss y accuracy
                # ──────────────────────────────────────────────────────────
                """plt.figure(figsize=(6,4))
                plt.title("Loss function during training")
                sns.lineplot(x=range(1, len(training_loss) + 1), y=training_loss, color="blue")
                plt.xlabel("Epoch #")
                plt.ylabel("Loss")
                plt.tight_layout()
                plt.show()

                plt.figure(figsize=(6,4))
                plt.title("Accuracy during training")
                sns.lineplot(x=range(1, len(training_accuracy) + 1), y=training_accuracy, color="orange")
                plt.xlabel("Epoch #")
                plt.ylabel("Accuracy")
                plt.tight_layout()
                plt.show()"""

                # ──────────────────────────────────────────────────────────
                # 2.6) Evaluación en test y métricas finales
                # ──────────────────────────────────────────────────────────
                y_test_np = np.asarray(y_test)
                y_pred    = vqc.predict(X_test_q)

                print(classification_report(
                    y_test_np,
                    y_pred,
                    target_names=["fail (0)", "success (1)"],
                    digits=4
                ))

                accuracy_test  = accuracy_score(y_test_np,   y_pred)
                # 3) Métricas promedio ponderado
                precision_w = precision_score(y_test_np, y_pred, average='macro')
                recall_w    = recall_score(   y_test_np, y_pred, average='macro')
                f1_w        = f1_score(       y_test_np, y_pred, average='macro')

                cm = confusion_matrix(y_test_np, y_pred)
                print("\nConfusion matrix [[TN, FP], [FN, TP]]:\n", cm)
                print("\n")

                grid_results3["feature_map"].append(fm_name)
                grid_results3["ansatz"].append(ans_name)
                grid_results3["rep_fm"].append(REPS_FM)
                grid_results3["rep_var"].append(REPS_ANS)
                grid_results3["learning_rate"].append(LR)
                grid_results3["loss_func"].append(loss_name)
                grid_results3["accuracy"].append(accuracy_test)
                grid_results3["precision"].append(precision_w)
                grid_results3["recall"].append(recall_w)
                grid_results3["f1"].append(f1_w)
                grid_results3["duration"].append(duration)

print("\nGrid results:\n")
evaluation3 = pd.DataFrame(grid_results3)

evaluation3 = evaluation3.sort_values(
    ["accuracy", "precision", "recall"],
    ascending=False
)

display(evaluation3)


══════════════════════════════════════════════════════════════════════════════════════════
State: 1/192:
══════════════════════════════════════════════════════════════════════════════════════════
		 Feature map: ZFeatureMap
		 Ansatz: EfficientSU2
		 Reps FM: 1
		 Reps var layer: 2
		 Learning rate: 0.001
		 Loss function: weightedcross_entropy
══════════════════════════════════════════════════════════════════════════════════════════

1   | loss 0.92398 | train acc 0.7662
2   | loss 0.75839 | train acc 0.8125
3   | loss 0.76273 | train acc 0.7940
4   | loss 0.80494 | train acc 0.8287
5   | loss 0.78272 | train acc 0.8009
6   | loss 0.77101 | train acc 0.7917
7   | loss 0.79369 | train acc 0.8565
8   | loss 0.85886 | train acc 0.6991
9   | loss 0.76716 | train acc 0.7847
10  | loss 0.79003 | train acc 0.7801
11  | loss 0.79080 | train acc 0.8079
12  | loss 0.77020 | train acc 0.7917
13  | loss 0.79948 | train acc 0.7593
14  | loss 0.89810 | train acc 0.6759
15  | loss 0.75839 | train ac

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

1   | loss 1.11332 | train acc 0.4213
2   | loss 1.04926 | train acc 0.5417
3   | loss 0.85861 | train acc 0.7130
4   | loss 0.85654 | train acc 0.7106
5   | loss 0.88846 | train acc 0.6736
6   | loss 0.62867 | train acc 0.8218
7   | loss 0.67724 | train acc 0.7986
8   | loss 1.00686 | train acc 0.4769
9   | loss 0.71796 | train acc 0.8102
10  | loss 0.66838 | train acc 0.8333
11  | loss 0.75590 | train acc 0.7824
12  | loss 0.75966 | train acc 0.8333
13  | loss 0.84172 | train acc 0.6944
14  | loss 0.73170 | train acc 0.8981
15  | loss 0.62867 | train acc 0.8218
16  | loss 0.62867 | train acc 0.8218
17  | loss 0.62867 | train acc 0.8218
18  | loss 0.62867 | train acc 0.8218
19  | loss 0.62867 | train acc 0.8218
20  | loss 0.56507 | train acc 0.8009
21  | loss 0.73951 | train acc 0.7708
22  | loss 0.63516 | train acc 0.8056
23  | loss 0.59164 | train acc 0.7801
24  | loss 0.58945 | train acc 0.7824
25  | loss 0.55243 | train acc 0.7986
26  | loss 0.56716 | train acc 0.7963
27  | loss 0

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

1   | loss 0.79204 | train acc 0.7940
2   | loss 0.69054 | train acc 0.7963
3   | loss 0.69159 | train acc 0.7801
4   | loss 0.69268 | train acc 0.7963
5   | loss 0.72981 | train acc 0.7500
6   | loss 0.69449 | train acc 0.7662
7   | loss 0.67643 | train acc 0.7940
8   | loss 1.24698 | train acc 0.2384
9   | loss 0.74191 | train acc 0.7569
10  | loss 0.68468 | train acc 0.7870
11  | loss 0.63720 | train acc 0.8426
12  | loss 0.65840 | train acc 0.8194
13  | loss 0.69263 | train acc 0.7870
14  | loss 0.53282 | train acc 0.9144
15  | loss 0.51746 | train acc 0.9144
16  | loss 0.51034 | train acc 0.9144
17  | loss 0.52249 | train acc 0.9144
18  | loss 0.51535 | train acc 0.9144
19  | loss 0.50514 | train acc 0.9144
20  | loss 0.56521 | train acc 0.9097
21  | loss 0.50514 | train acc 0.9144
22  | loss 0.50514 | train acc 0.9144
23  | loss 0.50514 | train acc 0.9144
24  | loss 0.50514 | train acc 0.9144
25  | loss 0.50514 | train acc 0.9144
26  | loss 0.46826 | train acc 0.9144
27  | loss 0

...

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

1   | loss 0.33267 | train acc 0.7315
2   | loss 0.51363 | train acc 0.5787
3   | loss 0.65497 | train acc 0.4074
4   | loss 0.54103 | train acc 0.5093
5   | loss 0.63857 | train acc 0.4537
6   | loss 0.35636 | train acc 0.6921
7   | loss 0.35712 | train acc 0.7130
8   | loss 0.76463 | train acc 0.3611
9   | loss 0.43205 | train acc 0.6574
10  | loss 0.22830 | train acc 0.9097
11  | loss 0.43791 | train acc 0.6088
12  | loss 0.22830 | train acc 0.9097
13  | loss 0.22830 | train acc 0.9097
14  | loss 0.57906 | train acc 0.4931
15  | loss 0.24311 | train acc 0.9028
16  | loss 0.20004 | train acc 0.9144
17  | loss 0.22307 | train acc 0.9144
18  | loss 0.19875 | train acc 0.9144
19  | loss 0.21628 | train acc 0.9005
20  | loss 0.18947 | train acc 0.9144
21  | loss 0.22082 | train acc 0.9190
22  | loss 0.19674 | train acc 0.9144
23  | loss 0.24661 | train acc 0.8796
24  | loss 0.20143 | train acc 0.9144
25  | loss 0.19568 | train acc 0.9144
26  | loss 0.18947 | train acc 0.9144
27  | loss 0

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

1   | loss 0.54169 | train acc 0.4144
2   | loss 0.56292 | train acc 0.4745
3   | loss 0.50939 | train acc 0.5139
4   | loss 0.48609 | train acc 0.5718
5   | loss 0.50620 | train acc 0.5532
6   | loss 0.43725 | train acc 0.6644
7   | loss 0.51414 | train acc 0.4745
8   | loss 0.43821 | train acc 0.6296
9   | loss 0.43766 | train acc 0.6921
10  | loss 0.41873 | train acc 0.7338
11  | loss 0.46177 | train acc 0.6458
12  | loss 0.43741 | train acc 0.7037
13  | loss 0.46145 | train acc 0.6620
14  | loss 0.36607 | train acc 0.7500
15  | loss 0.36607 | train acc 0.7500
16  | loss 0.36607 | train acc 0.7500
17  | loss 0.36607 | train acc 0.7500
18  | loss 0.36607 | train acc 0.7500
19  | loss 0.36607 | train acc 0.7500
20  | loss 0.39552 | train acc 0.7338
21  | loss 0.38527 | train acc 0.7222
22  | loss 0.44761 | train acc 0.6412
23  | loss 0.41156 | train acc 0.7060
24  | loss 0.37729 | train acc 0.7546
25  | loss 0.36945 | train acc 0.7593
26  | loss 0.38595 | train acc 0.7199
27  | loss 0

,feature_map,ansatz,rep_fm,rep_var,learning_rate,loss_func,accuracy,precision,recall,f1,duration
107,ZFeatureMap,RealAmplitudes,1,3,0.0001,absolute_error,0.935185,0.792500,0.762626,0.776530,8.149626
171,ZZFeatureMap,RealAmplitudes,1,3,0.0001,squared_error,0.916667,0.725000,0.702020,0.712681,9.297769
113,ZZFeatureMap,EfficientSU2,1,2,0.0001,absolute_error,0.916667,0.712264,0.550505,0.568958,9.771609
48,ZFeatureMap,EfficientSU2,1,2,0.0010,cross_entropy,0.916667,0.458333,0.500000,0.478261,8.589424
50,ZFeatureMap,EfficientSU2,1,3,0.0010,cross_entropy,0.916667,0.458333,0.500000,0.478261,9.130921
...,...,...,...,...,...,...,...,...,...,...,...
47,PauliFeatureMap,RealAmplitudes,2,3,0.0001,weightedcross_entropy,0.537037,0.560372,0.696970,0.454545,11.409453
34,PauliFeatureMap,EfficientSU2,1,3,0.0010,weightedcross_entropy,0.527778,0.575000,0.742424,0.456965,10.320743
45,PauliFeatureMap,RealAmplitudes,2,2,0.0001,weightedcross_entropy,0.527778,0.558966,0.691919,0.448262,11.171653
44,PauliFeatureMap,RealAmplitudes,2,2,0.0010,weightedcross_entropy,0.518519,0.541796,0.636364,0.432727,11.224395


In [8]:
from qiskit_machine_learning.optimizers import L_BFGS_B

weighted_loss_fn = WeightedCrossEntropyLoss(class_weight=class_weight_list)

# ──────────────────────────────────────────────────────────
# Parámetros de grid (solo L_BFGS_B)
# ──────────────────────────────────────────────────────────
params = {
    "loss_func_list"   : ["weightedcross_entropy", "cross_entropy", "absolute_error", "squared_error"],
    "feature_map_list" : ["ZFeatureMap", "ZZFeatureMap", "PauliFeatureMap"],
    "ansatz_list"      : ["EfficientSU2", "RealAmplitudes"],
    # repeticiones fijas
    "rep_fm_list"       : [1, 2],
    "rep_ans_list"      : [2, 3],
    # iteraciones para L_BFGS_B
    "lbfgs_maxiter"    : 10,
}

# mapeos de nombre a clase
feature_map_classes = {
    "ZFeatureMap"     : ZFeatureMap,
    "ZZFeatureMap"    : ZZFeatureMap,
    "PauliFeatureMap" : PauliFeatureMap,
}

ansatz_classes = {
    "EfficientSU2"   : EfficientSU2,
    "RealAmplitudes" : RealAmplitudes,
}

loss_fn_map = {
    "weightedcross_entropy": weighted_loss_fn,
    "cross_entropy":         "cross_entropy",
    "absolute_error":        "absolute_error",
    "squared_error":         "squared_error",
}

# inicializar resultados
grid_results2 = {
    "feature_map": [],
    "ansatz": [],
    "rep_fm": [],
    "rep_var": [],
    "learning_rate": [],
    "loss_func": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": [],
    "duration": [],
}

n_runs = (
    len(params["loss_func_list"])
    * len(params["feature_map_list"])
    * len(params["ansatz_list"])
    * len(params["rep_fm_list"])
    * len(params["rep_ans_list"])
)

actual_run = 0

# ──────────────────────────────────────────────────────────
# 2) Grid search loop
# ──────────────────────────────────────────────────────────
for loss_name in params["loss_func_list"]:
    for fm_name in params["feature_map_list"]:
        for ans_name in params["ansatz_list"]:
          for rep_fm in params["rep_fm_list"]:       # <— nuevo bucle
            for rep_ans in params["rep_ans_list"]: # <— nuevo bucle

              actual_run += 1

              N_QUBITS  = 3
              REPS_FM   = rep_fm
              REPS_ANS  = rep_ans
              EPOCHS    = params["lbfgs_maxiter"]

              # Banner informativo
              print("═"*90)
              print(f"State: {actual_run}/{n_runs}:")
              print("═"*90)
              print(f"\t\t Feature map: {fm_name}")
              print(f"\t\t Ansatz: {ans_name}")
              print(f"\t\t Reps FM: {REPS_FM}")
              print(f"\t\t Reps var layer: {REPS_ANS}")
              print(f"\t\t Loss function: {loss_name}")
              print("═"*90, end="\n\n")
              
              sampler_inst = AerSampler(run_options={"shots": None})
              loss_arg = loss_fn_map.get(loss_name)

              # Instanciar circuitos
              feature_map = feature_map_classes[fm_name](
                  feature_dimension=N_QUBITS, reps=REPS_FM
              )
              ansatz = ansatz_classes[ans_name](
                  num_qubits=N_QUBITS, reps=REPS_ANS
              )

              training_loss     = []
              training_accuracy = []
              epoch_log         = []

              def logger(weights, loss_val):
                  """Se ejecuta en cada paso del optimizador."""
                  epoch  = len(epoch_log) + 1
                  # forward() → probas, tomamos clase con mayor probabilidad
                  logits = vqc._neural_network.forward(X_train_q, weights)
                  preds  = np.argmax(logits, axis=1)
                  acc    = accuracy_score(y_train, preds)
                  training_loss.append(loss_val)
                  training_accuracy.append(acc)
                  epoch_log.append((epoch, loss_val, acc))
                  print(f"{epoch:<3} | loss {loss_val:.5f} | train acc {acc:.4f}")

              # Optimizer L_BFGS_B
              optimizer = L_BFGS_B(maxiter=EPOCHS)

              # Construir VQC
              vqc = VQC(
                  sampler      = sampler_inst,
                  feature_map  = feature_map,
                  ansatz       = ansatz,
                  optimizer    = optimizer,
                  loss         = loss_arg,
              )

              vqc.callback = logger                # enlazamos el callback

              # Entrenamiento
              start = time.time()
              vqc.fit(X_train_q, y_train)
              duration = time.time() - start
              print(
                  "\n\n---------> Training duration: "
                  + time.strftime("%H:%M:%S", time.gmtime(duration))
              )

              # Estadística final
              mean_acc = np.mean(training_accuracy)
              print(
                  f"\n\t\tMean training accuracy in the {EPOCHS} iterations: "
                  f"{mean_acc:.4f}\n"
              )

              # Plots de loss y accuracy
              """plt.figure(figsize=(6,4))
              plt.title("Loss function during training")
              sns.lineplot(
                  x=range(1, len(training_loss) + 1),
                  y=training_loss,
                  color="blue"
              )
              plt.xlabel("Iteration #"); plt.ylabel("Loss"); plt.tight_layout(); plt.show()

              plt.figure(figsize=(6,4))
              plt.title("Accuracy during training")
              sns.lineplot(
                  x=range(1, len(training_accuracy) + 1),
                  y=training_accuracy,
                  color="orange"
              )
              plt.xlabel("Iteration #"); plt.ylabel("Accuracy"); plt.tight_layout(); plt.show()
              """
              # Evaluación en test
              y_test_np = np.asarray(y_test)
              y_pred    = vqc.predict(X_test_q)

              print(
                  classification_report(
                      y_test_np,
                      y_pred,
                      target_names=["fail (0)", "success (1)"],
                      digits=4
                  )
              )

              accuracy_test  = accuracy_score(y_test_np,   y_pred)
              # 3) Métricas promedio ponderado
              precision_w = precision_score(y_test_np, y_pred, average='macro')
              recall_w    = recall_score(   y_test_np, y_pred, average='macro')
              f1_w        = f1_score(       y_test_np, y_pred, average='macro')

              cm = confusion_matrix(y_test_np, y_pred)
              print("\nConfusion matrix [[TN, FP], [FN, TP]]:\n", cm)
              print("\n")

              grid_results2["feature_map"].append(fm_name)
              grid_results2["ansatz"].append(ans_name)
              grid_results2["rep_fm"].append(REPS_FM)
              grid_results2["rep_var"].append(REPS_ANS)
              grid_results2["learning_rate"].append(LR)
              grid_results2["loss_func"].append(loss_name)
              grid_results2["accuracy"].append(accuracy_test)
              grid_results2["precision"].append(precision_w)
              grid_results2["recall"].append(recall_w)
              grid_results2["f1"].append(f1_w)
              grid_results2["duration"].append(duration)

# Mostrar resultados finales
print("\nGrid results:\n")
evaluation2 = pd.DataFrame(grid_results2)
evaluation2 = evaluation2.sort_values(
    ["accuracy", "precision", "recall"],
    ascending=False
)
display(evaluation2)


══════════════════════════════════════════════════════════════════════════════════════════
State: 1/96:
══════════════════════════════════════════════════════════════════════════════════════════
		 Feature map: ZFeatureMap
		 Ansatz: EfficientSU2
		 Reps FM: 1
		 Reps var layer: 2
		 Loss function: weightedcross_entropy
══════════════════════════════════════════════════════════════════════════════════════════

1   | loss 0.82027 | train acc 0.6528
2   | loss 0.73803 | train acc 0.8056
3   | loss 0.69203 | train acc 0.7569
4   | loss 0.77522 | train acc 0.5301
5   | loss 0.68357 | train acc 0.7500
6   | loss 0.67012 | train acc 0.6713
7   | loss 0.65949 | train acc 0.7153
8   | loss 0.65537 | train acc 0.7315
9   | loss 0.65158 | train acc 0.7384
10  | loss 0.64740 | train acc 0.7407
11  | loss 0.63554 | train acc 0.7917
12  | loss 0.61751 | train acc 0.8426


---------> Training duration: 00:01:38

		Mean training accuracy in the 10 iterations: 0.7272

              precision    recall

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

1   | loss 0.64413 | train acc 0.7917
2   | loss 0.51857 | train acc 0.8495
3   | loss 0.46960 | train acc 0.9028
4   | loss 0.43310 | train acc 0.9144
5   | loss 0.41453 | train acc 0.9190
6   | loss 0.38728 | train acc 0.9167
7   | loss 0.38512 | train acc 0.9167
8   | loss 0.37963 | train acc 0.9144
9   | loss 0.38037 | train acc 0.9167
10  | loss 0.37964 | train acc 0.9144
11  | loss 0.37963 | train acc 0.9144
12  | loss 0.37963 | train acc 0.9144
13  | loss 0.37963 | train acc 0.9144
14  | loss 0.37963 | train acc 0.9144
15  | loss 0.37963 | train acc 0.9144
16  | loss 0.37963 | train acc 0.9144
17  | loss 0.37963 | train acc 0.9144


---------> Training duration: 00:00:43

		Mean training accuracy in the 10 iterations: 0.9033

              precision    recall  f1-score   support

    fail (0)     0.0000    0.0000    0.0000         9
 success (1)     0.9167    1.0000    0.9565        99

    accuracy                         0.9167       108
   macro avg     0.4583    0.5000    0.

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

1   | loss 1.08986 | train acc 0.2894
2   | loss 0.86719 | train acc 0.6620
3   | loss 0.73105 | train acc 0.7708
4   | loss 0.58804 | train acc 0.8333
5   | loss 0.48607 | train acc 0.8634
6   | loss 0.45956 | train acc 0.9144
7   | loss 0.40624 | train acc 0.9144
8   | loss 0.39427 | train acc 0.9097
9   | loss 0.39144 | train acc 0.9144
10  | loss 0.38839 | train acc 0.9167
11  | loss 0.38457 | train acc 0.9144


---------> Training duration: 00:00:46

		Mean training accuracy in the 10 iterations: 0.8093

              precision    recall  f1-score   support

    fail (0)     0.0000    0.0000    0.0000         9
 success (1)     0.9167    1.0000    0.9565        99

    accuracy                         0.9167       108
   macro avg     0.4583    0.5000    0.4783       108
weighted avg     0.8403    0.9167    0.8768       108


Confusion matrix [[TN, FP], [FN, TP]]:
 [[ 0  9]
 [ 0 99]]


══════════════════════════════════════════════════════════════════════════════════════════
State

...

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

1   | loss 0.43667 | train acc 0.6551
2   | loss 0.36992 | train acc 0.7292
3   | loss 0.35075 | train acc 0.7477
4   | loss 0.34686 | train acc 0.7569
5   | loss 0.34446 | train acc 0.7685
6   | loss 0.34365 | train acc 0.7731
7   | loss 0.34321 | train acc 0.7708
8   | loss 0.34308 | train acc 0.7755
9   | loss 0.34279 | train acc 0.7685
10  | loss 0.34246 | train acc 0.7778
11  | loss 0.34243 | train acc 0.7778


---------> Training duration: 00:02:00

		Mean training accuracy in the 10 iterations: 0.7546

              precision    recall  f1-score   support

    fail (0)     0.1538    0.4444    0.2286         9
 success (1)     0.9390    0.7778    0.8508        99

    accuracy                         0.7500       108
   macro avg     0.5464    0.6111    0.5397       108
weighted avg     0.8736    0.7500    0.7990       108


Confusion matrix [[TN, FP], [FN, TP]]:
 [[ 4  5]
 [22 77]]


══════════════════════════════════════════════════════════════════════════════════════════
State

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

1   | loss 0.50066 | train acc 0.5602
2   | loss 0.44193 | train acc 0.6366
3   | loss 0.40316 | train acc 0.6921
4   | loss 0.56453 | train acc 0.4838
5   | loss 0.36699 | train acc 0.7315
6   | loss 0.36238 | train acc 0.7384
7   | loss 0.36024 | train acc 0.7384
8   | loss 0.35642 | train acc 0.7569
9   | loss 0.35315 | train acc 0.7593
10  | loss 0.34990 | train acc 0.7708
11  | loss 0.34705 | train acc 0.7847
12  | loss 0.34546 | train acc 0.7731


---------> Training duration: 00:02:12

		Mean training accuracy in the 10 iterations: 0.7022

              precision    recall  f1-score   support

    fail (0)     0.1250    0.3333    0.1818         9
 success (1)     0.9286    0.7879    0.8525        99

    accuracy                         0.7500       108
   macro avg     0.5268    0.5606    0.5171       108
weighted avg     0.8616    0.7500    0.7966       108


Confusion matrix [[TN, FP], [FN, TP]]:
 [[ 3  6]
 [21 78]]


══════════════════════════════════════════════════════════

,feature_map,ansatz,rep_fm,rep_var,learning_rate,loss_func,accuracy,precision,recall,f1,duration
26,ZFeatureMap,EfficientSU2,2,2,0.0001,cross_entropy,0.944444,0.971429,0.666667,0.735294,124.977359
74,ZFeatureMap,EfficientSU2,2,2,0.0001,squared_error,0.944444,0.971429,0.666667,0.735294,124.779091
24,ZFeatureMap,EfficientSU2,1,2,0.0001,cross_entropy,0.925926,0.962617,0.555556,0.580583,154.093456
25,ZFeatureMap,EfficientSU2,1,3,0.0001,cross_entropy,0.925926,0.962617,0.555556,0.580583,153.667878
73,ZFeatureMap,EfficientSU2,1,3,0.0001,squared_error,0.925926,0.962617,0.555556,0.580583,167.836393
...,...,...,...,...,...,...,...,...,...,...,...
94,PauliFeatureMap,RealAmplitudes,2,2,0.0001,squared_error,0.731481,0.521107,0.550505,0.505604,44.412176
14,ZZFeatureMap,RealAmplitudes,2,2,0.0001,weightedcross_entropy,0.648148,0.595745,0.808081,0.541964,73.403162
15,ZZFeatureMap,RealAmplitudes,2,3,0.0001,weightedcross_entropy,0.583333,0.552759,0.671717,0.475332,67.544570
23,PauliFeatureMap,RealAmplitudes,2,3,0.0001,weightedcross_entropy,0.564815,0.580357,0.762626,0.482832,85.553418


In [18]:
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import ADAM


# ──────────────────────────────────────────────────────────
# Parámetros de grid (solo ADAM)
# ──────────────────────────────────────────────────────────
params = {
    "loss_func_list"   : ["weightedcross_entropy", "cross_entropy", "absolute_error", "squared_error"],
    "feature_map_list"  : ["ZFeatureMap", "ZZFeatureMap", "PauliFeatureMap"],
    "ansatz_list"       : ["EfficientSU2", "RealAmplitudes"],
    "rep_fm_list"       : [1, 2],
    "rep_ans_list"      : [2, 3],
    # iteraciones para ADAM
    "adam_maxiter"      : 10,
    # tasas de aprendizaje a explorar
    "learning_rate_list": [0.01, 0.001],
}

# mapeos de nombre a clase
feature_map_classes = {
    "ZFeatureMap"     : ZFeatureMap,
    "ZZFeatureMap"    : ZZFeatureMap,
    "PauliFeatureMap" : PauliFeatureMap,
}

ansatz_classes = {
    "EfficientSU2"   : EfficientSU2,
    "RealAmplitudes" : RealAmplitudes,
}

loss_fn_map = {
    "weightedcross_entropy": weighted_loss_fn,
    "cross_entropy":         "cross_entropy",
    "absolute_error":        "absolute_error",
    "squared_error":         "squared_error",
}

# inicializar resultados
grid_results1 = {
    "feature_map": [],
    "ansatz": [],
    "rep_fm": [],
    "rep_var": [],
    "learning_rate": [],
    "loss_func": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": [],
    "duration": [],
}

n_runs = len(params["loss_func_list"]) * len(params["feature_map_list"]) * \
         len(params["ansatz_list"]) * len(params["learning_rate_list"]   * \
         len(params["rep_fm_list"])  * len(params["rep_ans_list"]))

actual_run = 0

# ──────────────────────────────────────────────────────────
# 2) Grid search loop
# ──────────────────────────────────────────────────────────
for loss_name in params["loss_func_list"]:
    for fm_name in params["feature_map_list"]:
        for ans_name in params["ansatz_list"]:
          for rep_fm in params["rep_fm_list"]:
            for rep_ans in params["rep_ans_list"]:
              for LR in params["learning_rate_list"]:

                  actual_run += 1
                  N_QUBITS  = 3
                  REPS_FM   = rep_fm
                  REPS_ANS  = rep_ans
                  EPOCHS    = params["adam_maxiter"]

                  # Banner informativo
                  print("═"*90)
                  print(f"State: {actual_run}/{n_runs}:")
                  print("═"*90)
                  print(f"\t\t Feature map: {fm_name}")
                  print(f"\t\t Ansatz: {ans_name}")
                  print(f"\t\t Reps FM: {REPS_FM}")
                  print(f"\t\t Reps var layer: {REPS_ANS}")
                  print(f"\t\t Learning rate: {LR}")
                  print(f"\t\t Loss function: {loss_name}")
                  print("═"*90, end="\n\n")

                  loss_arg = loss_fn_map.get(loss_name)
                  sampler_inst = AerSampler(run_options={"shots": None})

                  # Instanciar circuitos
                  feature_map = feature_map_classes[fm_name](feature_dimension=N_QUBITS, reps=REPS_FM)
                  ansatz      = ansatz_classes[ans_name](num_qubits=N_QUBITS, reps=REPS_ANS)

                  training_loss     = []
                  training_accuracy = []
                  epoch_log         = []

                  def logger(step, weights, loss, **kwargs):
                      """Callback de ADAM: registra al final de cada iteración."""
                      epoch = len(training_loss) + 1
                      preds = np.argmax(vqc._neural_network.forward(X_train_q, weights), axis=1)
                      acc   = accuracy_score(y_train, preds)
                      training_loss.append(loss)
                      training_accuracy.append(acc)
                      epoch_log.append((epoch, loss, acc))
                      print(f"{epoch:>3} | loss={loss:.4f} | acc={acc:.3f}")

                  # Optimizer ADAM
                  optimizer = ADAM(
                      maxiter=EPOCHS,
                      lr=LR,
                      callback=logger
                  )

                  # Construir VQC
                  vqc = VQC(
                      sampler      = sampler_inst,
                      feature_map  = feature_map,
                      ansatz       = ansatz,
                      optimizer    = optimizer,
                      loss         = loss_arg,
                  )

                  # Entrenamiento
                  start = time.time()
                  vqc.fit(X_train_q, y_train)
                  duration = time.time() - start
                  print("\n\n---------> Training duration: {}".format(
                      time.strftime("%H:%M:%S", time.gmtime(duration))
                  ))

                  # Estadística final
                  mean_acc = np.mean(training_accuracy)
                  print(f"\n\t\tMean training accuracy in the {EPOCHS} epochs: {mean_acc:.4f}\n")

                  # Plots de loss y accuracy
                  """plt.figure(figsize=(6,4))
                  plt.title("Loss function during training")
                  sns.lineplot(x=range(1, len(training_loss) + 1), y=training_loss, color="blue")
                  plt.xlabel("Epoch #"); plt.ylabel("Loss"); plt.tight_layout(); plt.show()

                  plt.figure(figsize=(6,4))
                  plt.title("Accuracy during training")
                  sns.lineplot(x=range(1, len(training_accuracy) + 1), y=training_accuracy, color="orange")
                  plt.xlabel("Epoch #"); plt.ylabel("Accuracy"); plt.tight_layout(); plt.show()
                  """

                  # Evaluación en test
                  y_test_np = np.asarray(y_test)
                  y_pred    = vqc.predict(X_test_q)

                  print(classification_report(
                      y_test_np, y_pred,
                      target_names=["fail (0)", "success (1)"],
                      digits=4
                  ))

                  accuracy_test  = accuracy_score(y_test_np,   y_pred)
                  # 3) Métricas promedio ponderado (weighted average)
                  precision_w = precision_score(y_test_np, y_pred, average='macro')
                  recall_w    = recall_score(   y_test_np, y_pred, average='macro')
                  f1_w        = f1_score(       y_test_np, y_pred, average='macro')

                  cm = confusion_matrix(y_test_np, y_pred)
                  print("\nConfusion matrix [[TN, FP], [FN, TP]]:\n", cm)
                  print("\n")

                  grid_results1["feature_map"].append(fm_name)
                  grid_results1["ansatz"].append(ans_name)
                  grid_results1["rep_fm"].append(REPS_FM)
                  grid_results1["rep_var"].append(REPS_ANS)
                  grid_results1["learning_rate"].append(LR)
                  grid_results1["loss_func"].append(loss_name)
                  grid_results1["accuracy"].append(accuracy_test)
                  grid_results1["precision"].append(precision_w)
                  grid_results1["recall"].append(recall_w)
                  grid_results1["f1"].append(f1_w)
                  grid_results1["duration"].append(duration)

# Mostrar resultados finales
print("\nGrid results:\n")
evaluation1 = pd.DataFrame(grid_results1)
evaluation1 = evaluation1.sort_values(
    ["accuracy", "precision", "recall"],
    ascending=False
)
display(evaluation1)


══════════════════════════════════════════════════════════════════════════════════════════
State: 1/192:
══════════════════════════════════════════════════════════════════════════════════════════
		 Feature map: ZFeatureMap
		 Ansatz: EfficientSU2
		 Reps FM: 1
		 Reps var layer: 2
		 Learning rate: 0.01
		 Loss function: weightedcross_entropy
══════════════════════════════════════════════════════════════════════════════════════════

  1 | loss=0.9662 | acc=0.481
  2 | loss=0.9553 | acc=0.514
  3 | loss=0.9448 | acc=0.567
  4 | loss=0.9346 | acc=0.623
  5 | loss=0.9248 | acc=0.676
  6 | loss=0.9154 | acc=0.713
  7 | loss=0.9063 | acc=0.748
  8 | loss=0.8976 | acc=0.762
  9 | loss=0.8892 | acc=0.775
 10 | loss=0.8811 | acc=0.787


---------> Training duration: 00:01:20

		Mean training accuracy in the 10 epochs: 0.6646

              precision    recall  f1-score   support

    fail (0)     0.2857    0.6667    0.4000         9
 success (1)     0.9655    0.8485    0.9032        99

    a

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

  1 | loss=1.0331 | acc=0.602
  2 | loss=1.0276 | acc=0.604
  3 | loss=1.0223 | acc=0.606
  4 | loss=1.0172 | acc=0.606
  5 | loss=1.0123 | acc=0.604
  6 | loss=1.0075 | acc=0.602
  7 | loss=1.0029 | acc=0.597
  8 | loss=0.9985 | acc=0.602
  9 | loss=0.9942 | acc=0.604
 10 | loss=0.9901 | acc=0.604


---------> Training duration: 00:01:31

		Mean training accuracy in the 10 epochs: 0.6032

              precision    recall  f1-score   support

    fail (0)     0.0769    0.3333    0.1250         9
 success (1)     0.9130    0.6364    0.7500        99

    accuracy                         0.6111       108
   macro avg     0.4950    0.4848    0.4375       108
weighted avg     0.8434    0.6111    0.6979       108


Confusion matrix [[TN, FP], [FN, TP]]:
 [[ 3  6]
 [36 63]]


══════════════════════════════════════════════════════════════════════════════════════════
State: 18/192:
══════════════════════════════════════════════════════════════════════════════════════════
		 Feature map: ZZFea

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

  1 | loss=0.6727 | acc=0.861
  2 | loss=0.6584 | acc=0.875
  3 | loss=0.6456 | acc=0.882
  4 | loss=0.6345 | acc=0.880
  5 | loss=0.6248 | acc=0.889
  6 | loss=0.6167 | acc=0.896
  7 | loss=0.6099 | acc=0.898
  8 | loss=0.6045 | acc=0.898
  9 | loss=0.6002 | acc=0.898
 10 | loss=0.5968 | acc=0.898


---------> Training duration: 00:00:47

		Mean training accuracy in the 10 epochs: 0.8875

              precision    recall  f1-score   support

    fail (0)     0.0000    0.0000    0.0000         9
 success (1)     0.9167    1.0000    0.9565        99

    accuracy                         0.9167       108
   macro avg     0.4583    0.5000    0.4783       108
weighted avg     0.8403    0.9167    0.8768       108


Confusion matrix [[TN, FP], [FN, TP]]:
 [[ 0  9]
 [ 0 99]]


══════════════════════════════════════════════════════════════════════════════════════════
State: 64/192:
══════════════════════════════════════════════════════════════════════════════════════════
		 Feature map: ZFeat

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/m

  1 | loss=0.7046 | acc=0.812
  2 | loss=0.7033 | acc=0.812
  3 | loss=0.7021 | acc=0.819
  4 | loss=0.7008 | acc=0.822
  5 | loss=0.6995 | acc=0.824
  6 | loss=0.6983 | acc=0.824
  7 | loss=0.6970 | acc=0.829
  8 | loss=0.6958 | acc=0.829
  9 | loss=0.6946 | acc=0.831
 10 | loss=0.6934 | acc=0.836


---------> Training duration: 00:00:47

		Mean training accuracy in the 10 epochs: 0.8238

              precision    recall  f1-score   support

    fail (0)     0.2000    0.1111    0.1429         9
 success (1)     0.9223    0.9596    0.9406        99

    accuracy                         0.8889       108
   macro avg     0.5612    0.5354    0.5417       108
weighted avg     0.8621    0.8889    0.8741       108


Confusion matrix [[TN, FP], [FN, TP]]:
 [[ 1  8]
 [ 4 95]]


══════════════════════════════════════════════════════════════════════════════════════════
State: 65/192:
══════════════════════════════════════════════════════════════════════════════════════════
		 Feature map: ZZFea

,feature_map,ansatz,rep_fm,rep_var,learning_rate,loss_func,accuracy,precision,recall,f1,duration
108,ZFeatureMap,RealAmplitudes,2,2,0.010,absolute_error,0.916667,0.712264,0.550505,0.568958,29.674894
15,ZFeatureMap,RealAmplitudes,2,3,0.001,weightedcross_entropy,0.916667,0.458333,0.500000,0.478261,47.881788
61,ZFeatureMap,RealAmplitudes,2,2,0.001,cross_entropy,0.916667,0.458333,0.500000,0.478261,29.510805
62,ZFeatureMap,RealAmplitudes,2,3,0.010,cross_entropy,0.916667,0.458333,0.500000,0.478261,47.688925
51,ZFeatureMap,EfficientSU2,1,3,0.001,cross_entropy,0.907407,0.457944,0.494949,0.475728,139.854775
...,...,...,...,...,...,...,...,...,...,...,...
43,PauliFeatureMap,RealAmplitudes,1,3,0.001,weightedcross_entropy,0.342593,0.431818,0.287879,0.274894,49.715440
151,ZFeatureMap,EfficientSU2,2,3,0.001,squared_error,0.305556,0.529550,0.570707,0.287912,151.454599
4,ZFeatureMap,EfficientSU2,2,2,0.010,weightedcross_entropy,0.296296,0.419490,0.262626,0.245588,89.180920
55,ZFeatureMap,EfficientSU2,2,3,0.001,cross_entropy,0.287037,0.401880,0.207071,0.231636,152.343158


In [19]:
print("\n Optimizador SPSA:")
display(evaluation)
print("\n Optimizador COBYLA:")
display(evaluation3)
print("\n\n Optimizador L_BFGS_B:")
display(evaluation2)
print("\n Optimizador ADAM:")
display(evaluation1)


 Optimizador SPSA:


,feature_map,ansatz,rep_fm,rep_var,learning_rate,loss_func,accuracy,precision,recall,f1,duration
146,ZFeatureMap,EfficientSU2,1,3,0.05,squared_error,0.944444,0.875728,0.717172,0.770863,13.647756
50,ZFeatureMap,EfficientSU2,1,3,0.05,weightedcross_entropy,0.925926,0.758704,0.858586,0.797563,13.833588
51,ZFeatureMap,EfficientSU2,1,3,0.10,weightedcross_entropy,0.925926,0.758704,0.858586,0.797563,13.960713
10,ZFeatureMap,RealAmplitudes,1,3,0.05,cross_entropy,0.916667,0.712264,0.550505,0.568958,12.317431
1,ZFeatureMap,EfficientSU2,1,2,0.10,cross_entropy,0.916667,0.458333,0.500000,0.478261,12.891154
...,...,...,...,...,...,...,...,...,...,...,...
88,PauliFeatureMap,RealAmplitudes,1,2,0.05,weightedcross_entropy,0.509259,0.478276,0.429293,0.382058,13.556021
90,PauliFeatureMap,RealAmplitudes,1,3,0.05,weightedcross_entropy,0.472222,0.518750,0.560606,0.393079,13.897192
138,PauliFeatureMap,RealAmplitudes,1,3,0.05,absolute_error,0.472222,0.472222,0.409091,0.361345,14.047859
171,ZZFeatureMap,RealAmplitudes,1,3,0.10,squared_error,0.472222,0.472222,0.409091,0.361345,14.007853



 Optimizador COBYLA:


,feature_map,ansatz,rep_fm,rep_var,learning_rate,loss_func,accuracy,precision,recall,f1,duration
107,ZFeatureMap,RealAmplitudes,1,3,0.0001,absolute_error,0.935185,0.792500,0.762626,0.776530,8.149626
171,ZZFeatureMap,RealAmplitudes,1,3,0.0001,squared_error,0.916667,0.725000,0.702020,0.712681,9.297769
113,ZZFeatureMap,EfficientSU2,1,2,0.0001,absolute_error,0.916667,0.712264,0.550505,0.568958,9.771609
48,ZFeatureMap,EfficientSU2,1,2,0.0010,cross_entropy,0.916667,0.458333,0.500000,0.478261,8.589424
50,ZFeatureMap,EfficientSU2,1,3,0.0010,cross_entropy,0.916667,0.458333,0.500000,0.478261,9.130921
...,...,...,...,...,...,...,...,...,...,...,...
47,PauliFeatureMap,RealAmplitudes,2,3,0.0001,weightedcross_entropy,0.537037,0.560372,0.696970,0.454545,11.409453
34,PauliFeatureMap,EfficientSU2,1,3,0.0010,weightedcross_entropy,0.527778,0.575000,0.742424,0.456965,10.320743
45,PauliFeatureMap,RealAmplitudes,2,2,0.0001,weightedcross_entropy,0.527778,0.558966,0.691919,0.448262,11.171653
44,PauliFeatureMap,RealAmplitudes,2,2,0.0010,weightedcross_entropy,0.518519,0.541796,0.636364,0.432727,11.224395




 Optimizador L_BFGS_B:


,feature_map,ansatz,rep_fm,rep_var,learning_rate,loss_func,accuracy,precision,recall,f1,duration
26,ZFeatureMap,EfficientSU2,2,2,0.0001,cross_entropy,0.944444,0.971429,0.666667,0.735294,124.977359
74,ZFeatureMap,EfficientSU2,2,2,0.0001,squared_error,0.944444,0.971429,0.666667,0.735294,124.779091
24,ZFeatureMap,EfficientSU2,1,2,0.0001,cross_entropy,0.925926,0.962617,0.555556,0.580583,154.093456
25,ZFeatureMap,EfficientSU2,1,3,0.0001,cross_entropy,0.925926,0.962617,0.555556,0.580583,153.667878
73,ZFeatureMap,EfficientSU2,1,3,0.0001,squared_error,0.925926,0.962617,0.555556,0.580583,167.836393
...,...,...,...,...,...,...,...,...,...,...,...
94,PauliFeatureMap,RealAmplitudes,2,2,0.0001,squared_error,0.731481,0.521107,0.550505,0.505604,44.412176
14,ZZFeatureMap,RealAmplitudes,2,2,0.0001,weightedcross_entropy,0.648148,0.595745,0.808081,0.541964,73.403162
15,ZZFeatureMap,RealAmplitudes,2,3,0.0001,weightedcross_entropy,0.583333,0.552759,0.671717,0.475332,67.544570
23,PauliFeatureMap,RealAmplitudes,2,3,0.0001,weightedcross_entropy,0.564815,0.580357,0.762626,0.482832,85.553418



 Optimizador ADAM:


,feature_map,ansatz,rep_fm,rep_var,learning_rate,loss_func,accuracy,precision,recall,f1,duration
108,ZFeatureMap,RealAmplitudes,2,2,0.010,absolute_error,0.916667,0.712264,0.550505,0.568958,29.674894
15,ZFeatureMap,RealAmplitudes,2,3,0.001,weightedcross_entropy,0.916667,0.458333,0.500000,0.478261,47.881788
61,ZFeatureMap,RealAmplitudes,2,2,0.001,cross_entropy,0.916667,0.458333,0.500000,0.478261,29.510805
62,ZFeatureMap,RealAmplitudes,2,3,0.010,cross_entropy,0.916667,0.458333,0.500000,0.478261,47.688925
51,ZFeatureMap,EfficientSU2,1,3,0.001,cross_entropy,0.907407,0.457944,0.494949,0.475728,139.854775
...,...,...,...,...,...,...,...,...,...,...,...
43,PauliFeatureMap,RealAmplitudes,1,3,0.001,weightedcross_entropy,0.342593,0.431818,0.287879,0.274894,49.715440
151,ZFeatureMap,EfficientSU2,2,3,0.001,squared_error,0.305556,0.529550,0.570707,0.287912,151.454599
4,ZFeatureMap,EfficientSU2,2,2,0.010,weightedcross_entropy,0.296296,0.419490,0.262626,0.245588,89.180920
55,ZFeatureMap,EfficientSU2,2,3,0.001,cross_entropy,0.287037,0.401880,0.207071,0.231636,152.343158


In [11]:
evaluation.to_pickle('evaluation.pkl')
evaluation2.to_pickle('evaluation2.pkl')
evaluation3.to_pickle('evaluation3.pkl')

In [13]:
evaluation.to_csv('evaluation.csv', index=False)
evaluation2.to_csv('evaluation2.csv', index=False)
evaluation3.to_csv('evaluation3.csv', index=False)

In [21]:
evaluation1.to_pickle('evaluation1.pkl')
evaluation1.to_csv('evaluation1.csv', index=False)

In [14]:
threshold = 0.55

# 1) filtrar para quedarnos sólo con los modelos cuyo recall ≥ threshold
eval_filtrado = evaluation[evaluation['recall'] >= threshold]

# 2) ordenar ese filtrado por accuracy descendente
eval_ordenado = eval_filtrado.sort_values(
    by='accuracy',
    ascending=False
)

print(eval_ordenado.head(10))

     feature_map          ansatz  rep_fm  rep_var  learning_rate  \
146  ZFeatureMap    EfficientSU2       1        3           0.05   
50   ZFeatureMap    EfficientSU2       1        3           0.05   
51   ZFeatureMap    EfficientSU2       1        3           0.10   
10   ZFeatureMap  RealAmplitudes       1        3           0.05   
99   ZFeatureMap    EfficientSU2       1        3           0.10   
154  ZFeatureMap  RealAmplitudes       1        3           0.05   
48   ZFeatureMap    EfficientSU2       1        2           0.05   
107  ZFeatureMap  RealAmplitudes       1        3           0.10   
49   ZFeatureMap    EfficientSU2       1        2           0.10   
104  ZFeatureMap  RealAmplitudes       1        2           0.05   

                 loss_func  accuracy  precision    recall        f1   duration  
146          squared_error  0.944444   0.875728  0.717172  0.770863  13.647756  
50   weightedcross_entropy  0.925926   0.758704  0.858586  0.797563  13.833588  
51   wei

In [15]:
threshold = 0.55

# 1) filtrar para quedarnos sólo con los modelos cuyo recall ≥ threshold
eval_filtrado2 = evaluation2[evaluation2['recall'] >= threshold]

# 2) ordenar ese filtrado por accuracy descendente
eval_ordenado2 = eval_filtrado2.sort_values(
    by='accuracy',
    ascending=False
)

print(eval_ordenado2.head(10))

     feature_map          ansatz  rep_fm  rep_var  learning_rate  \
26   ZFeatureMap    EfficientSU2       2        2         0.0001   
74   ZFeatureMap    EfficientSU2       2        2         0.0001   
24   ZFeatureMap    EfficientSU2       1        2         0.0001   
25   ZFeatureMap    EfficientSU2       1        3         0.0001   
73   ZFeatureMap    EfficientSU2       1        3         0.0001   
85  ZZFeatureMap  RealAmplitudes       1        3         0.0001   
2    ZFeatureMap    EfficientSU2       2        2         0.0001   
0    ZFeatureMap    EfficientSU2       1        2         0.0001   
3    ZFeatureMap    EfficientSU2       2        3         0.0001   
7    ZFeatureMap  RealAmplitudes       2        3         0.0001   

                loss_func  accuracy  precision    recall        f1    duration  
26          cross_entropy  0.944444   0.971429  0.666667  0.735294  124.977359  
74          squared_error  0.944444   0.971429  0.666667  0.735294  124.779091  
24      

In [16]:
threshold = 0.55

# 1) filtrar para quedarnos sólo con los modelos cuyo recall ≥ threshold
eval_filtrado3 = evaluation3[evaluation3['recall'] >= threshold]

# 2) ordenar ese filtrado por accuracy descendente
eval_ordenado3 = eval_filtrado3.sort_values(
    by='accuracy',
    ascending=False
)

print(eval_ordenado3.head(10))

         feature_map          ansatz  rep_fm  rep_var  learning_rate  \
107      ZFeatureMap  RealAmplitudes       1        3         0.0001   
171     ZZFeatureMap  RealAmplitudes       1        3         0.0001   
113     ZZFeatureMap    EfficientSU2       1        2         0.0001   
14       ZFeatureMap  RealAmplitudes       2        3         0.0010   
49       ZFeatureMap    EfficientSU2       1        2         0.0001   
6        ZFeatureMap    EfficientSU2       2        3         0.0010   
178  PauliFeatureMap    EfficientSU2       1        3         0.0010   
7        ZFeatureMap    EfficientSU2       2        3         0.0001   
104      ZFeatureMap  RealAmplitudes       1        2         0.0010   
64      ZZFeatureMap    EfficientSU2       1        2         0.0010   

                 loss_func  accuracy  precision    recall        f1   duration  
107         absolute_error  0.935185   0.792500  0.762626  0.776530   8.149626  
171          squared_error  0.916667   0.7250

In [22]:
threshold = 0.55

# 1) filtrar para quedarnos sólo con los modelos cuyo recall ≥ threshold
eval_filtrado1 = evaluation1[evaluation1['recall'] >= threshold]

# 2) ordenar ese filtrado por accuracy descendente
eval_ordenado1 = eval_filtrado1.sort_values(
    by='accuracy',
    ascending=False
)

print(eval_ordenado1.head(10))

         feature_map          ansatz  rep_fm  rep_var  learning_rate  \
108      ZFeatureMap  RealAmplitudes       2        2          0.010   
48       ZFeatureMap    EfficientSU2       1        2          0.010   
178  PauliFeatureMap    EfficientSU2       1        3          0.010   
96       ZFeatureMap    EfficientSU2       1        2          0.010   
145      ZFeatureMap    EfficientSU2       1        2          0.001   
56       ZFeatureMap  RealAmplitudes       1        2          0.010   
0        ZFeatureMap    EfficientSU2       1        2          0.010   
155      ZFeatureMap  RealAmplitudes       1        3          0.001   
104      ZFeatureMap  RealAmplitudes       1        2          0.010   
2        ZFeatureMap    EfficientSU2       1        3          0.010   

                 loss_func  accuracy  precision    recall        f1  \
108         absolute_error  0.916667   0.712264  0.550505  0.568958   
48           cross_entropy  0.870370   0.627530  0.676768  0.6457